In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
import json
import requests
import datetime
from datetime import timedelta
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import HTML

In [4]:
start_date = input('Please enter the start date for the investment in the YYYY-MM-DD format: ')
date_format = '%Y-%m-%d'
while True:
    try:
        datetime.datetime.strptime(start_date, date_format)
        break
    except ValueError:
        print('Provided date is not in correct format')
        start_date = input('Please enter the start date for the investment in the YYYY-MM-DD format: ')

Please enter the start date for the investment in the YYYY-MM-DD format:  2020/09/20


Provided date is not in correct format


Please enter the start date for the investment in the YYYY-MM-DD format:  2020-09-20


In [5]:
year, month, day = map(int, start_date.split('-'))
next_date = datetime.date(year, month, day) + timedelta(30)
end_date = next_date.strftime('%Y') + '-' + next_date.strftime('%m') + '-' + next_date.strftime('%d') 
nbp_import = requests.get('http://api.nbp.pl/api/exchangerates/tables/A/{stdate}/{endate}/'.format(stdate = str(start_date), endate = str(end_date)))
ccy_series = nbp_import.json()
table_frame = pd.DataFrame(ccy_series)
uncut_rates = table_frame['rates']
rates = [pd.DataFrame(line).set_index('code') for line in uncut_rates]
ccy_series = (pd.concat([rates[table]['mid'] for table in range(0, len(rates))], axis=1).transpose()).set_index(table_frame['effectiveDate'])
headers = [header for header in ccy_series]

In [7]:
request_1 = 'Please enter the first currency (ISO format) for the investment: '
request_2 = 'Please enter the second currency (ISO format) for the investment: '
request_3 = 'Please enter the third currency (ISO format) for the investment: '

response_1 = 'Currency is not in ISO format'
response_2 = 'Unfortunately, we do not have such currency in our database'
response_3 = 'You have already selected this currency'
CCY_IN1 = str(input(request_1))
CCY_IN1 = CCY_IN1.upper()
while len(CCY_IN1) != 3:
    print(response_1)
    CCY_IN1 = str(input(request_1))
    CCY_IN1 = CCY_IN1.upper()
while CCY_IN1 not in headers:
    print(response_2)
    CCY_IN1 = str(input(request_1))
    CCY_IN1 = CCY_IN1.upper()
CCY_IN2 = str(input(request_2))
CCY_IN2 = CCY_IN2.upper()
while len(CCY_IN2) != 3:
    print(response_1)
    CCY_IN2 = str(input(request_2))
    CCY_IN2 = CCY_IN2.upper()
while CCY_IN2 == CCY_IN1:
    print(response_3)
    CCY_IN2 = str(input(request_2))
    CCY_IN2 = CCY_IN2.upper()
while CCY_IN2 not in headers:
    print(response_2)
    CCY_IN2 = str(input(request_2))
    CCY_IN2 = CCY_IN2.upper()
CCY_IN3 = str(input(request_3))
CCY_IN3 = CCY_IN3.upper()
while len(CCY_IN3) != 3:
    print(response_1)
    CCY_IN3 = str(input(request_3))
    CCY_IN3 = CCY_IN3.upper()
while CCY_IN3 in [CCY_IN1, CCY_IN2]:
    print(response_3)
    CCY_IN3 = str(input(request_3))
    CCY_IN3 = CCY_IN3.upper()
while CCY_IN3 not in headers:
    print(response_2)
    CCY_IN3 = str(input(request_3))
    CCY_IN3 = CCY_IN3.upper()

ccy = [str(CCY_IN1), str(CCY_IN2), str(CCY_IN3)]
ccy_series[ccy]

Please enter the first currency (ISO format) for the investment:  usd
Please enter the second currency (ISO format) for the investment:  eur
Please enter the third currency (ISO format) for the investment:  gbp


code,USD,EUR,GBP
effectiveDate,,,
2020-09-21,3.7963,4.4800,4.8759
2020-09-22,3.8359,4.5055,4.9195
2020-09-23,3.8399,4.4946,4.8785
2020-09-24,3.8833,4.5272,4.9467
2020-09-25,3.9016,4.5477,4.9802
2020-09-28,3.9083,4.5497,5.0121
2020-09-29,3.9228,4.5834,5.0522
2020-09-30,3.8658,4.5268,4.9560
2020-10-01,3.8513,4.5153,4.9391


In [16]:
amp1 = widgets.FloatSlider(min=0, max=100, step=0.5, value=100/3, description=str(CCY_IN1),
                           readout_format ='.1f', orientation='vertical')
amp2 = widgets.FloatSlider(min=0, max=100, step=0.5, value=100/3, description=str(CCY_IN2), 
                           readout_format ='.1f', orientation='vertical')
amp3 = widgets.FloatSlider(min=0, max=100, step=0.5, value=100/3, description=str(CCY_IN3), 
                           readout_format ='.1f', orientation='vertical')

def update_slider1(change):
    amp2.value = 100 - amp1.value - amp3.value
def update_slider2(change):
    amp3.value = 100 - amp2.value - amp1.value
def update_slider3(change):
    amp1.value = 100 - amp3.value - amp2.value
    
amp1.observe(update_slider1, 'value')
amp2.observe(update_slider2, 'value')
amp3.observe(update_slider3, 'value')

display(widgets.HBox([amp1, amp2, amp3]))

In [ ]:
amp1.value + amp2.value + amp3.value

In [ ]:
checkbox2.value

In [ ]:
def update_slider1(change):
    amp2.value = 1 - amp1.value - amp3.value
def update_slider2(change):
    amp3.value = 1 - amp2.value - amp1.value
def update_slider3(change):
    amp1.value = 1 - amp3.value - amp2.value
    
amp1.observe(update_slider1, 'value')
amp2.observe(update_slider2, 'value')
amp3.observe(update_slider3, 'value')

In [ ]:
from ipywidgets import interact
def f(x):
    return x
interact(f, x = widgets.IntSlider)